In [1]:
# Parameters
execution_date = "2023-05-26T00:00:00+00:00"


In [2]:
from delta import configure_spark_with_delta_pip
from delta.tables import DeltaTable
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [3]:
builder = SparkSession.builder.master("spark://spark-master:7077").appName("curatedApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

/home/airflow/.local/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/home/airflow/.local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/airflow/.ivy2/cache
The jars for the packages stored in: /home/airflow/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-30a3ff6a-2e59-435e-8c2d-f22cafa52e7a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 114ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

23/05/27 04:06:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
df = spark.read.format('delta').load('/opt/workspace/data/cleaned/iris/')

23/05/27 04:06:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df = (
    df
    .groupBy('Species')
    .agg({
        'SepalLengthCm': 'sum', 
        'SepalWidthCm': 'sum', 
        'PetalLengthCm': 'sum', 
        'PetalWidthCm': 'sum'}
    ).withColumn('sum_petal_widhcm', col('sum(PetalWidthCm)').cast('decimal(12, 2)'))
    .withColumn('sum_sepal_widhcm', col('sum(SepalWidthCm)').cast('decimal(12, 2)'))
    .withColumn('sum_petal_lengthcm', col('sum(PetalLengthCm)').cast('decimal(12, 2)'))
    .withColumn('sum_sepal_lengthcm', col('sum(SepalLengthCm)').cast('decimal(12, 2)'))
    .select(
        'sum_petal_widhcm',
        'sum_sepal_widhcm',
        'sum_petal_lengthcm',
        'sum_sepal_lengthcm'
    )
)

In [6]:
df.write.mode('overwrite').format('delta').save('/opt/workspace/data/curated/iris/')

In [7]:
spark.stop()